In [51]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)

from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import tensor_basis,spinful_fermion_basis_1d,boson_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library
#
##### setting up parameters for simulation
# physical parameters
l = 1
L_f = 3 # system size f
L_b = L_f-1 # system size b
Nf, Nb = 2, l # number of fermions, bosons
N = Nf + Nb # total number of particles
# define time-dependent perturbation

###### create the basis
# build the two bases to tensor together to a bose-fermi mixture
basis_b1=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_b2=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_f=spinful_fermion_basis_1d(L_f,Nf = [(0,0),(0,1),(1,0),(1,1),(2,0),(0,2),(1,2),(2,1),(2,2),(0,3),(3,0),(3,1),(1,3),(2,3),(3,2),(3,3)],double_occupancy=True)
basis=tensor_basis(basis_b1,basis_b2,basis_f) # BFM

############################################## target  hamiltonian
a = 0.1
e = 1.0
m = 0.3
lam = 0.5

bosonic_part_LGT = [[a*e**2/4.,i,i,i,i] for i in range(L_b)] # b^dagger*b^dagger*b*b
mass_f = [[m+1.0/a,i,i] for i in range(L_f)] # f mass term
m_mass_f = [[-m-1.0/a,i,i] for i in range(L_f)] # f mass term

interaction_part_LGT = [[1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction
m_interaction_part_LGT = [[-1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction

gauge_violation = [[lam,i,(i+1)] for i in range(L_f-1)] #gauge violating term
m_gauge_violation = [[-lam,i,(i+1)] for i in range(L_f-1)] #gauge violating term


b_0 = [[-0.5*e,0]]
d_0 = [[+0.5*e,0]]
b_1 = [[+0.5*e,1]]
d_1 = [[-0.5*e,1]]
rho_1 = [[-e,1]]
eee = [[e,1]]

static_gauss_law = [
    
    ["n|||", b_0],
    ["|n||", d_0],
    ["n|||", b_1],
    ["|n||", d_1],
    ["||n|", rho_1],
    ["|||n", rho_1],
    ["|I||", eee]

]    


    
    

#
static_target = [
            ["++--|||", bosonic_part_LGT], # b^dagger*b^dagger*b*b
            ["|++--||", bosonic_part_LGT], # b^dagger*b^dagger*b*b

            ["||+|-", mass_f], # f mass term
            ["||-|+", m_mass_f], # f mass term


            ["+|-|+|-", interaction_part_LGT], #boson-fermion interaction
            ["-|+|-|+", m_interaction_part_LGT], #boson-fermion interaction


]
static_violation = [
["||+|-", gauge_violation],
["||-|+", m_gauge_violation]

]

################################################
###### set up Hamiltonian and initial states
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

H_violation = hamiltonian(static_violation,dynamic_list = [],basis=basis,**no_checks)
H_target = hamiltonian(static_target,dynamic_list = [],basis=basis,**no_checks)
Gauss_law = hamiltonian(static_gauss_law, dynamic_list = [], basis=basis, **no_checks)



In [52]:
# define initial Fock state through strings
s_f = ["111","110","101","011","100","010","001","000"]
s_b = ["20","11","10","02","01","00"]
# basis.index accepts strings and returns the index which corresponds to that state in the basis list
i = np.zeros(8)
i[0] = basis.index(s_b[2],s_b[2],s_f[0],s_f[7]) # find index of product state in basis
i[1] = basis.index(s_b[2],s_b[2],s_f[1],s_f[6])
i[2] = basis.index(s_b[2],s_b[2],s_f[2],s_f[5])
i[3] = basis.index(s_b[2],s_b[2],s_f[3],s_f[4])
i[4] = basis.index(s_b[2],s_b[2],s_f[4],s_f[3])
i[5] = basis.index(s_b[2],s_b[2],s_f[5],s_f[2])
i[6] = basis.index(s_b[2],s_b[2],s_f[6],s_f[1])
i[7] = basis.index(s_b[2],s_b[2],s_f[7],s_f[0])


psi_0 = np.zeros(basis.Ns) # allocate space for state
psi_0[int(i[0])] = 1/(np.sqrt(2)**3) # set MB state to be the given product state
psi_0[int(i[1])] = -1/(np.sqrt(2)**3)
psi_0[int(i[2])] = -1/(np.sqrt(2)**3)
psi_0[int(i[3])] = -1/(np.sqrt(2)**3)
psi_0[int(i[4])] = 1/(np.sqrt(2)**3)
psi_0[int(i[5])] = 1/(np.sqrt(2)**3)
psi_0[int(i[6])] = 1/(np.sqrt(2)**3)
psi_0[int(i[7])] = -1/(np.sqrt(2)**3)


#


In [53]:
#eigenvalues,eigenvectors = H_target.eigh()
#psi_ground = eigenvectors[:,0]
###### time evolve initial state and measure energy
import datetime

times = np.linspace(0.0,10,100)
def extract_energy(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_0,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

print(datetime.datetime.now())
extract_energy(np.array([10.,10.]))
print(datetime.datetime.now())


2021-10-22 15:11:19.338961
2021-10-22 15:11:20.429113


In [54]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())


2021-10-22 15:11:28.954852
2021-10-22 15:14:09.495269


In [55]:
print(resbrute[0]) #theta1, theta2

[3.12793052 8.18047511]


In [56]:
print(resbrute[1]) #minimized energy

-4.506222272442281


In [49]:
Gauss_law.__init__(static_gauss_law, dynamic_list = [],basis=basis, check_herm = True)
#for checking the Hermiticity of the operators

Hermiticity check passed!


C:\Users\pavka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_symm=False in hamiltonian
  """Entry point for launching an IPython kernel.
C:\Users\pavka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Test for particle conservation not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_pcon=False in hamiltonian
  """Entry point for launching an IPython kernel.


In [63]:
Gauss_law.expt_value(psi_0)

0j

In [62]:
times1=np.linspace(0.0,resbrute[0][0],100)
times = np.linspace(0.0,10.,100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_0,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
#print(np.abs(np.dot(psi_ground,np.array([row[-1] for row in psi_t1]))))
#print(np.abs(np.dot(np.array([row[-1] for row in psi_t1]),np.array([row[-1] for row in psi_t2]))))
#print(np.abs(np.dot(np.array([row[-1] for row in psi_t2]),np.array([row[-1] for row in psi_t3]))))
Gauss_law.expt_value([row[-1] for row in psi_t3])**2

(1.6197070832296654e-29+0j)